In [1]:
import pandas as pd
import numpy as np
import os
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('Datasets//movies_metadata.csv')

C:\Users\Nihar\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [5]:
df.fillna('', inplace=True)

In [29]:
columns_needed = ['genres', 'overview', 'title']
df_modified = df[columns_needed]
df_modified

,genres,overview,title
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...",Toy Story
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,Jumanji
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...,Grumpier Old Men
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale
4,"[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...,Father of the Bride Part II
...,...,...,...
45461,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",Rising and falling between a man and woman.,Subdue
45462,"[{'id': 18, 'name': 'Drama'}]",An artist struggles to finish his work while a...,Century of Birthing
45463,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...","When one of her hits goes wrong, a professiona...",Betrayal
45464,[],"In a small town live two brothers, one a minis...",Satan Triumphant


In [30]:
df_modified['genres'] = df_modified['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

C:\Users\Nihar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
df_modified.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genres    45466 non-null  object
 1   overview  45466 non-null  object
 2   title     45466 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


In [32]:
def combine_genres(col):
    genres = ''
    for i in range(len(col)):
        genres = genres + ' ' + col[i]
    return genres

In [33]:
df_modified['combined_genres'] = df_modified['genres'].apply(combine_genres)

C:\Users\Nihar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
df_modified.drop('genres', axis=1, inplace=True)

C:\Users\Nihar\Anaconda3\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [35]:
df_modified

,overview,title,combined_genres
0,"Led by Woody, Andy's toys live happily in his ...",Toy Story,Animation Comedy Family
1,When siblings Judy and Peter discover an encha...,Jumanji,Adventure Fantasy Family
2,A family wedding reignites the ancient feud be...,Grumpier Old Men,Romance Comedy
3,"Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale,Comedy Drama Romance
4,Just when George Banks has recovered from his ...,Father of the Bride Part II,Comedy
...,...,...,...
45461,Rising and falling between a man and woman.,Subdue,Drama Family
45462,An artist struggles to finish his work while a...,Century of Birthing,Drama
45463,"When one of her hits goes wrong, a professiona...",Betrayal,Action Drama Thriller
45464,"In a small town live two brothers, one a minis...",Satan Triumphant,


In [39]:
df_final = df_modified.copy()

In [40]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   overview         45466 non-null  object
 1   title            45466 non-null  object
 2   combined_genres  45466 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


In [41]:
def combine_frames(row):
    comb=''
    comb = row['overview'] + row['combined_genres']
    #comb = row['title'] + ' ' + row['overview'] + ' ' + row['tagline'] + ' ' + row['combined_genres']
    df_final['combined_data'] = comb

In [42]:
combine_frames(df_final)
#df_modified['combined_data'] = df_modified.apply(combine_frames)

In [43]:
df_final

,overview,title,combined_genres,combined_data
0,"Led by Woody, Andy's toys live happily in his ...",Toy Story,Animation Comedy Family,"Led by Woody, Andy's toys live happily in his ..."
1,When siblings Judy and Peter discover an encha...,Jumanji,Adventure Fantasy Family,When siblings Judy and Peter discover an encha...
2,A family wedding reignites the ancient feud be...,Grumpier Old Men,Romance Comedy,A family wedding reignites the ancient feud be...
3,"Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale,Comedy Drama Romance,"Cheated on, mistreated and stepped on, the wom..."
4,Just when George Banks has recovered from his ...,Father of the Bride Part II,Comedy,Just when George Banks has recovered from his ...
...,...,...,...,...
45461,Rising and falling between a man and woman.,Subdue,Drama Family,Rising and falling between a man and woman. Dr...
45462,An artist struggles to finish his work while a...,Century of Birthing,Drama,An artist struggles to finish his work while a...
45463,"When one of her hits goes wrong, a professiona...",Betrayal,Action Drama Thriller,"When one of her hits goes wrong, a professiona..."
45464,"In a small town live two brothers, one a minis...",Satan Triumphant,,"In a small town live two brothers, one a minis..."


In [44]:
df_part1 = df_final[:11366]
df_part1

,overview,title,combined_genres,combined_data
0,"Led by Woody, Andy's toys live happily in his ...",Toy Story,Animation Comedy Family,"Led by Woody, Andy's toys live happily in his ..."
1,When siblings Judy and Peter discover an encha...,Jumanji,Adventure Fantasy Family,When siblings Judy and Peter discover an encha...
2,A family wedding reignites the ancient feud be...,Grumpier Old Men,Romance Comedy,A family wedding reignites the ancient feud be...
3,"Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale,Comedy Drama Romance,"Cheated on, mistreated and stepped on, the wom..."
4,Just when George Banks has recovered from his ...,Father of the Bride Part II,Comedy,Just when George Banks has recovered from his ...
...,...,...,...,...
11361,The mastermind behind a ubiquitous spy operati...,Spies,Action Drama Thriller,The mastermind behind a ubiquitous spy operati...
11362,"Dito Montiel, a successful author, receives a ...",A Guide To Recognizing Your Saints,Crime Drama,"Dito Montiel, a successful author, receives a ..."
11363,Young assassins Azumi and Nagara continue thei...,Azumi 2: Death or Love,Action,Young assassins Azumi and Nagara continue thei...
11364,"Sebastian, a young man, has decided to follow ...",13 Tzameti,Action Thriller,"Sebastian, a young man, has decided to follow ..."


In [45]:
df_part2 = df_final[11366:22732]
df_part2

,overview,title,combined_genres,combined_data
11366,The true story of anti-apartheid activists in ...,Catch a Fire,Action Drama Thriller,The true story of anti-apartheid activists in ...
11367,A fictional investigative documentary looks ba...,Death of a President,Documentary Drama Science Fiction,A fictional investigative documentary looks ba...
11368,"Emily Lindstrom, 14, is an aspiring concert vi...",Little Secrets,Drama Family,"Emily Lindstrom, 14, is an aspiring concert vi..."
11369,Diederich Heßling is scared of everything and ...,The Kaiser's Lackey,History Drama Comedy,Diederich Heßling is scared of everything and ...
11370,At the end of 1918 while civil war is raging o...,Corto Maltese in Siberia,Drama Animation History Adventure,At the end of 1918 while civil war is raging o...
...,...,...,...,...
22727,Aiden (Josh Lawson) craves a better life. A li...,Crave,Drama Thriller,Aiden (Josh Lawson) craves a better life. A li...
22728,"Screen superstars Clark Gable (""Gone With The ...",Betrayed,Drama Romance Thriller War,"Screen superstars Clark Gable (""Gone With The ..."
22729,"Rocky and Dan, war buddies, are prowl car cops...",Between Midnight and Dawn,Crime Drama,"Rocky and Dan, war buddies, are prowl car cops..."
22730,The continuation of Joe's sexually dictated li...,Nymphomaniac: Vol. II,Drama Mystery,The continuation of Joe's sexually dictated li...


In [46]:
df_part3 = df_final[22732:34098]
df_part3

,overview,title,combined_genres,combined_data
22732,A Swedish family found murdered in her home on...,A Place in the Sun,Crime Drama Thriller,A Swedish family found murdered in her home on...
22733,Blanche Dubois goes to visit her pregnant sist...,A Streetcar Named Desire,,Blanche Dubois goes to visit her pregnant sist...
22734,Blanche Dubois goes to visit her pregnant sist...,A Streetcar Named Desire,,Blanche Dubois goes to visit her pregnant sist...
22735,Steve is a talented gymnast who has given up c...,American Anthem,Romance,Steve is a talented gymnast who has given up c...
22736,A New York couple and their Midwestern friends...,Between Us,Drama,A New York couple and their Midwestern friends...
...,...,...,...,...
34093,John and Tina meet in a park one day. They imm...,Four Times That Night,Comedy,John and Tina meet in a park one day. They imm...
34094,A total re-edit of Mario Bava's gothic classic...,The House of Exorcism,Horror,A total re-edit of Mario Bava's gothic classic...
34095,A mysterious knife-throwing viking warrior pro...,Knives of the Avenger,Action Adventure,A mysterious knife-throwing viking warrior pro...
34096,A drifter helps a ranch couple fend off the ap...,Gunman Called Nebraska,Western,A drifter helps a ranch couple fend off the ap...


In [47]:
df_part4 = df_final[34098:45464]
df_part4

,overview,title,combined_genres,combined_data
34098,A liberal-thinking author watches his wife as ...,One Night at Dinner,Drama,A liberal-thinking author watches his wife as ...
34099,The career of screenwriter Dalton Trumbo is ha...,Trumbo,Drama,The career of screenwriter Dalton Trumbo is ha...
34100,A businessman comes up with a scheme to make m...,Panic Button,Comedy,A businessman comes up with a scheme to make m...
34101,,Two Sons of Ringo,Comedy,Comedy
34102,"Two famous gunmen, Lord and Bull are called to...",The Moment to Kill,Western,"Two famous gunmen, Lord and Bull are called to..."
...,...,...,...,...
45459,It's the year 3000 AD. The world's most danger...,Caged Heat 3000,Science Fiction,It's the year 3000 AD. The world's most danger...
45460,"Yet another version of the classic epic, with ...",Robin Hood,Drama Action Romance,"Yet another version of the classic epic, with ..."
45461,Rising and falling between a man and woman.,Subdue,Drama Family,Rising and falling between a man and woman. Dr...
45462,An artist struggles to finish his work while a...,Century of Birthing,Drama,An artist struggles to finish his work while a...


In [22]:
cv = CountVectorizer()

count_matrix1 = cv.fit_transform(df_part1["combined_data"])
count_matrix2 = cv.fit_transform(df_part2["combined_data"])
count_matrix3 = cv.fit_transform(df_part3["combined_data"])
count_matrix4 = cv.fit_transform(df_part4["combined_data"])

The next Cell used tfid instead of count vectorizer

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1, 1), min_df=0.0001, stop_words='english')

count_matrix1 = tfidf.fit_transform(df_part1["combined_data"])
count_matrix2 = tfidf.fit_transform(df_part2["combined_data"])
count_matrix3 = tfidf.fit_transform(df_part3["combined_data"])
count_matrix4 = tfidf.fit_transform(df_part4["combined_data"])

In [23]:
count_matrix1.dtype

dtype('int64')

Decreasing size of count matrix

In [24]:
count_matrix1 = count_matrix1.astype(np.int32)
count_matrix2 = count_matrix2.astype(np.int32)
count_matrix3 = count_matrix3.astype(np.int32)
count_matrix4 = count_matrix4.astype(np.int32)

In [49]:
cosine_sim1 = cosine_similarity(count_matrix1)
cosine_sim2 = cosine_similarity(count_matrix2)
cosine_sim3 = cosine_similarity(count_matrix3)
cosine_sim4 = cosine_similarity(count_matrix4)

In [50]:
combined = np.vstack([cosine_sim1, cosine_sim2, cosine_sim3, cosine_sim4])

In [51]:
def get_title_from_index(index):
    return df_final[df_final.index==index]['title'].values[0]
def get_index_from_title(title):
    return df_final[df_final.title==title].index[0]

In [67]:
movie_user_likes = "3 Idiots"

In [68]:
movie_index = get_index_from_title(movie_user_likes)
movie_index

14732

In [69]:
similar_movies = list(enumerate(combined[movie_index]))

In [70]:
sorted_similar_movies = sorted(similar_movies, key = lambda x:x[1], reverse = True)[1:]

In [71]:
i=0
print("Top 5 similar movies to "+movie_user_likes+" are:\n")
for element in sorted_similar_movies:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>10:
        break

Top 5 similar movies to 3 Idiots are:

Wuthering Heights
Whispering Corridors
The Woods
Chase a Crooked Shadow
From Dusk Till Dawn
City By The Sea
Grand Prix
Around the World in 80 Days
What's Up, Doc?
Fat Man and Little Boy
Space Cowboys


In [45]:
df_final[df_final.index==1773]

,title,overview,tagline,combined_genres,combined_data
1773,Godzilla,When a freighter is viciously attacked in the ...,Size does matter.,Science Fiction Action Thriller,Godzilla When a freighter is viciously attacke...


In [188]:
df_final[df_final.title=="Two Evil Eyes"]

,title,overview,tagline,combined_genres,combined_data
6259,Two Evil Eyes,Two horror segments based on Edgar Allan Poe s...,When I wake you... you'll be dead,Horror Thriller,Two Evil Eyes Two horror segments based on Edg...


In [72]:
np.save('Cosine_matrix1.csv', combined)